In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd
import gc
import ctypes
import matplotlib.pyplot as plt
import seaborn as sns

# TensorFlow and Keras for deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K, Model, Input, layers
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv1D, BatchNormalization, ReLU, MaxPooling1D, GlobalAveragePooling1D, Add, Concatenate, Input, LayerNormalization, MultiHeadAttention, Flatten, Reshape, LeakyReLU, Activation, Lambda, Layer
from tensorflow.keras.backend import clear_session
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError, Metric
from tensorflow.keras.optimizers import Adam, RMSprop, Adamax
from tensorflow.keras.callbacks import EarlyStopping, Callback, LearningRateScheduler
from tensorflow.keras.mixed_precision import Policy, set_global_policy
from tensorflow.keras.regularizers import l2, l1_l2
from tensorflow.data import Dataset
from keras import mixed_precision

# Scikit-learn for machine learning utilities
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler as sklearn_StandardScaler

# PySpark for distributed data processing
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, monotonically_increasing_id
from pyspark.sql.types import ArrayType, FloatType, IntegerType, DoubleType, StringType
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler as SparkVectorAssembler, MinMaxScaler as SparkMinMaxScaler, RobustScaler, StandardScaler as SparkStandardScaler, PCA as SparkPCA
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, DenseVector
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Keras Tuner for hyperparameter tuning
import keras_tuner as kt
from keras_tuner import HyperParameters, Hyperband

# Optuna for hyperparameter optimization
import optuna
from optuna.integration import TFKerasPruningCallback

# Numba for CUDA GPU acceleration
from numba import cuda

# Plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Logging information for dynamic error handling
import logging

# Console output coloring
from termcolor import colored

# Import sqrt function from math library
from math import sqrt

# Import count function from itertools library
from itertools import count
# Import mean, stddev, col, when functions from pyspark.sql.functions library
from pyspark.sql.functions import mean, stddev, col, when

In [3]:
# Set mixed precision policy to improve performance
# Mixed precision uses both 16-bit and 32-bit floating point types
# This can speed up training and reduce memory usage on compatible hardware
policy = Policy('bfloat16')
set_global_policy(policy)


In [ ]:
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

In [ ]:
# Initialize Spark session for the master node
# 1 - Use all available local cores for the master node
# 2 - Set the application name for the master node
# 3 - Configure the driver and executor memory and GPU settings
import subprocess
import json

# Configure Spark with GPU settings for RTX 4090 (24GB) and A5000 (24GB)
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ERROR 418 - I'M A TEAPOT") \
    .config("spark.driver.memory", "50g") \
    .config("spark.executor.memory", "50g") \
    .config("spark.driver.maxResultSize", "50g") \
    .config("spark.rapids.sql.enabled", "true") \
    .config("spark.rapids.memory.pinnedPool.size", "24G") \
    .config("spark.rapids.sql.concurrentGpuTasks", "2") \
    .config("spark.rapids.memory.gpu.pooling.enabled", "true") \
    .config("spark.rapids.memory.gpu.allocFraction", "0.95") \
    .config("spark.rapids.sql.explain", "ALL") \
    .config("spark.executor.resource.gpu.amount", "2") \
    .config("spark.task.resource.gpu.amount", "0.25") \
    .config("spark.rapids.sql.incompatibleOps.enabled", "true") \
    .config("spark.rapids.memory.host.spillStorageSize", "64G") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.adaptive.localShuffleReader.enabled", "true") \
    .config("spark.rapids.memory.gpu.maxAllocFraction", "0.95") \
    .config("spark.rapids.sql.batchSizeBytes", "512M") \
    .config("spark.rapids.sql.reader.batchSizeRows", "100000") \
    .config("spark.rapids.sql.variableRowGroupSize.enabled", "true") \
    .getOrCreate()

gpus = tf.config.list_logical_devices('GPU')
if gpus:
  # Replicate your computation on multiple GPUs
  c = []
  for gpu in gpus:
    with tf.device(gpu.name):
      a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
      b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
      c.append(tf.matmul(a, b))

  with tf.device('/CPU:0'):
    matmul_sum = tf.add_n(c)

  print(matmul_sum)

# Set log level to INFO
spark.sparkContext.setLogLevel("INFO")
# Configure logging
log4jLogger = spark._jvm.org.apache.log4j
logger = log4jLogger.LogManager.getLogger(__name__)

def custom_logger(level, message):
    color = 'white'
    if level == "INFO":
        color = 'cyan'
    elif level == "SUCCESS":
        color = 'green'
    elif level == "ERROR":
        color = 'red'
    elif level == "ACTION":
        color = 'blue'
    elif level == "PROGRESS" or level == "WARNING":
        color = 'yellow'
    elif level == "FINAL":
        color = 'magenta'
    logger.info(colored(f"SPARK: {level} - {message}", color))

# Set the environment variable for memory allocator
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1"

def initialize_cuda():
    cuda = ctypes.CDLL('libcuda.so')
    cuInit = cuda.cuInit
    cuInit.restype = ctypes.c_int
    cuInit.argtypes = [ctypes.c_uint]
    cuDeviceGetCount = cuda.cuDeviceGetCount
    cuDeviceGetCount.restype = ctypes.c_int
    cuDeviceGetCount.argtypes = [ctypes.POINTER(ctypes.c_int)]
    cuDeviceGet = cuda.cuDeviceGet
    cuDeviceGet.restype = ctypes.c_int
    cuDeviceGet.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]

    res = cuInit(0)
    if res != 0:
        raise RuntimeError("Failed to initialize CUDA")
    device_count = ctypes.c_int()
    res = cuDeviceGetCount(ctypes.byref(device_count))
    if res != 0:
        raise RuntimeError("Failed to get device count")
    devices = []
    for i in range(device_count.value):
        device = ctypes.c_int()
        res = cuDeviceGet(ctypes.byref(device), i)
        if res != 0:
            raise RuntimeError(f"Failed to get device {i}")
        devices.append(device.value)
    return devices

def allocate_gpu_memory(size):
    cuda = ctypes.CDLL('libcuda.so')
    cuda_malloc = cuda.cuMemAlloc
    cuda_malloc.restype = ctypes.c_int
    cuda_malloc.argtypes = [ctypes.POINTER(ctypes.c_ulonglong), ctypes.c_ulonglong]
    ptr = ctypes.c_ulonglong()
    res = cuda_malloc(ctypes.byref(ptr), size)
    if res != 0:
        raise RuntimeError("Failed to allocate GPU memory")
    return ptr.value

def free_gpu_memory(ptr):
    cuda = ctypes.CDLL('libcuda.so')
    cuda_free = cuda.cuMemFree
    cuda_free.restype = ctypes.c_int
    cuda_free.argtypes = [ctypes.c_ulonglong]
    res = cuda_free(ptr)
    if res != 0:
        raise RuntimeError("Failed to free GPU memory")

# Check GPU memory usage
def check_gpu_memory():
    os.system('nvidia-smi')

# Initialize CUDA
cuda_devices = initialize_cuda()

# Check available GPU memory
check_gpu_memory()

# Ensure TensorFlow uses the GPU by setting memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(f"Error setting memory growth: {e}")

# Check and print the number of GPUs available
num_gpus = len(tf.config.list_physical_devices('GPU'))
print(f"Num GPUs Available: {num_gpus}")

# Perform a simple TensorFlow operation to verify GPU usage
with tf.device('/GPU:0'):
    a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=tf.float32)
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]], dtype=tf.float32)
    c = tf.matmul(a, b)
print(f"Result of matrix multiplication on GPU:\n{c.numpy()}")

def get_gpu_memory_usage():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        memory_used = int(result.stdout.decode('utf-8').strip())
        return memory_used
    except Exception as e:
        print(f"Failed to get GPU memory usage: {e}")
        return None

def freegpu():
    try:
        # Get GPU memory usage before freeing
        memory_before = get_gpu_memory_usage()

        result = subprocess.run(['sudo', 'fuser', '-v', '/dev/nvidia*'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in result.stdout.decode('utf-8').split('\n'):
            if '/dev/nvidia' in line:
                pid = int(line.split()[-1])
                os.kill(pid, 9)

        # Get GPU memory usage after freeing
        memory_after = get_gpu_memory_usage()

        if memory_before is not None and memory_after is not None:
            memory_freed = memory_before - memory_after
            print(f"GPU memory has been freed. {memory_freed} MB of GPU memory was released.")
        else:
            print("GPU memory has been freed.")
    except Exception as e:
        print(f"Failed to free GPU memory: {e}")

# Set the environment variable for XLA flags
os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'

# ======================================================================================================
# ==============================[SETUP MIRROREDSTRATEGY WITHOUT NCCL]===================================
# ======================================================================================================

# ------------------------------------------------------------------------------------------------------
# Define the strategy with HierarchicalCopyAllReduce to avoid NCCL
# ------------------------------------------------------------------------------------------------------
strategy = tf.distribute.MirroredStrategy(
    devices=["/gpu:0", "/gpu:1"],
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()
)

print(f"Number of devices under strategy: {strategy.num_replicas_in_sync}")

### UNIVARIATE ANALYSIS

In [ ]:
# Load the processed data for autoencoder training
original_data_train = pd.read_csv('/home/skander/datasets/kaggle/compet/mental_health_2024/mental_test.csv', index_col=False, header=0)
original_data_test = pd.read_csv('/home/skander/datasets/kaggle/compet/mental_health_2024/mental_train.csv', index_col=False, header=0)

original_data = pd.concat([original_data_train, original_data_test], ignore_index=True)

custom_logger("INFO", "ORIGINAL DATA - BEFORE PROCESSING")

# Univariate analysis
def univariate_analysis(df):
    print("Univariate Analysis\n" + "="*40)
    
    # Shape
    print(f"Shape:\n{df.shape}\n")
    
    # Data Types
    print(f"Data Types:\n{df.dtypes}\n")
    
    # Missing Values
    missing_values = df.isnull().sum()
    print(f"Missing Values:\n{missing_values}\n")
    
    # Duplicates
    print(f"Duplicates:\n{df.duplicated().sum()}\n")
    
    # Summary statistics for numerical columns
    print("Summary Statistics for Numerical Columns:\n")
    print(df.describe(include=[np.number]))
    
    # Detailed information about NaN values
    print("\nDetailed Information about NaN Values:\n")
    for col in df.columns:
        if df[col].isnull().any():
            print(f"Column '{col}' has {df[col].isnull().sum()} NaN values, which is {df[col].isnull().mean() * 100:.2f}% of the column.")

# Perform univariate analysis
univariate_analysis(original_data)


# Univariate Analysis Observations and Insights

### Observations:
1. **Shape**: The dataset contains 234,500 rows and 20 columns.
2. **Data Types**: The dataset includes a mix of numerical and categorical data types.
3. **Missing Values**: Several columns have a significant number of missing values:
    - 'Academic Pressure', 'CGPA', and 'Study Satisfaction' have 80.10% missing values.
    - 'Depression' has 40.00% missing values.
    - 'Profession' and 'Job Satisfaction' have around 20% missing values.
4. **Duplicates**: There are no duplicate rows in the dataset.
5. **Summary Statistics**:
    - The 'Age' column ranges from 18 to 60, with a mean of approximately 40.36.
    - 'Work/Study Hours' has a wide range from 0 to 12 hours, with a mean of 6.25 hours.
    - 'Depression' is a binary column with a mean of 0.18, indicating a low prevalence in the dataset.
6. **NaN Values**: Columns with significant NaN values may need imputation or removal depending on their importance.

### Insights:
- The high percentage of missing values in 'Academic Pressure', 'CGPA', and 'Study Satisfaction' suggests these columns may not be reliable for analysis without imputation.
- The dataset appears to be balanced in terms of age distribution, but further analysis is needed to understand the distribution of other categorical variables like 'Gender' and 'City'.
- The 'Depression' column, being binary, can be used as a target variable for classification tasks, but missing values need to be addressed.
- The dataset's lack of duplicates is beneficial for analysis, ensuring data integrity.

### Recommendations:
- Consider imputing missing values for critical columns or removing them if they are not essential.
- Explore the categorical variables further to understand their distribution and potential impact on the target variable.
- Investigate the relationship between 'Work/Study Hours' and 'Depression' to identify any potential correlations.

### Next Steps:
- Visualize the distribution of key variables to gain further insights.
- Perform bivariate analysis to explore relationships between variables.
- Address missing values through imputation or removal based on their significance to the analysis.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from pyspark.sql import SparkSession

def segment_populations(df):
    # Students: Have academic-related data
    students = df[df['CGPA'].notna()].copy()
    
    # Workers: Have work-related data but no academic data
    workers = df[
        (df['Work Pressure'].notna()) & 
        (df['Job Satisfaction'].notna()) & 
        (df['CGPA'].isna())
    ].copy()
    
    # Unemployed: No profession but also not students
    unemployed = df[
        (df['Profession'].isna()) & 
        (df['CGPA'].isna()) & 
        (df['Work Pressure'].isna())
    ].copy()
    
    return students, workers, unemployed

def prepare_features(df):
    # Convert categorical variables
    le = LabelEncoder()
    categorical_cols = ['Gender', 'City', 'Sleep Duration', 'Dietary Habits', 
                       'Have you ever had suicidal thoughts ?', 
                       'Family History of Mental Illness', 'Profession']
    
    # Create numeric DataFrame for correlation analysis
    df_numeric = df.copy()
    
    for col in categorical_cols:
        if col in df.columns and df[col].notna().any():
            df_numeric[col] = le.fit_transform(df[col].astype(str))
    
    # Convert remaining numeric columns
    numeric_cols = df_numeric.select_dtypes(include=['object']).columns
    for col in numeric_cols:
        try:
            df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
        except:
            print(f"Could not convert {col} to numeric")
    
    return df_numeric

def analyze_segment(df, segment_name):
    print(f"\nAnalyzing {segment_name} Population:")
    print(f"Population size: {len(df)}")
    
    # Basic statistics
    print("\nNumerical Features Summary:")
    print(df.describe())
    
    # Correlation matrix for numeric columns only
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        plt.figure(figsize=(12, 8))
        sns.heatmap(df[numeric_cols].corr(), annot=True, cmap='coolwarm', center=0)
        plt.title(f'Correlation Matrix - {segment_name}')
        plt.tight_layout()
        plt.show()
    
    # Distribution plots for key metrics
    plot_distributions(df, segment_name)

def plot_distributions(df, segment_name):
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    n_cols = min(3, len(numeric_cols))
    if n_cols > 0:
        fig, axes = plt.subplots(1, n_cols, figsize=(15, 5))
        if n_cols == 1:
            axes = [axes]
        
        for i, col in enumerate(numeric_cols[:n_cols]):
            sns.histplot(data=df, x=col, ax=axes[i])
            axes[i].set_title(f'{col} Distribution')
        
        plt.suptitle(f'{segment_name} Population - Key Metrics Distribution')
        plt.tight_layout()
        plt.show()

# Main execution
students, workers, unemployed = segment_populations(original_data)

# Prepare features for each segment
students_prepared = prepare_features(students)
workers_prepared = prepare_features(workers)
unemployed_prepared = prepare_features(unemployed)

# Analyze each segment
analyze_segment(students_prepared, "Student")
analyze_segment(workers_prepared, "Worker")
analyze_segment(unemployed_prepared, "Unemployed")

# Plot depression rates across segments
def plot_depression_rates():
    plt.figure(figsize=(10, 6))
    
    segments = {
        'Students': students,
        'Workers': workers,
        'Unemployed': unemployed
    }
    
    depression_rates = []
    for name, segment in segments.items():
        rate = segment['Depression'].mean() * 100
        depression_rates.append({'Segment': name, 'Rate': rate})
    
    depression_df = pd.DataFrame(depression_rates)
    sns.barplot(x='Segment', y='Rate', data=depression_df)
    plt.title('Depression Rates Across Population Segments')
    plt.ylabel('Depression Rate (%)')
    plt.show()

plot_depression_rates()

# Convert pandas DataFrames to Spark DataFrames
students_spark = spark.createDataFrame(students)
workers_spark = spark.createDataFrame(workers) 
unemployed_spark = spark.createDataFrame(unemployed)

# Comprehensive Multivariate Analysis: Population Segments

## 1. Student Population (n=46,664)

### Key Demographics
- **Age**: Mean 25.8 years (SD=4.9), primarily young adults
- **Gender**: 55.7% one gender category
- **Academic Status**: All students (100%)

### Mental Health Profile
- **Depression Rate**: 58.6% (highest among segments)
- **Suicidal Thoughts**: 63.1% reported having suicidal thoughts
- **Family History**: 48.4% reported mental illness history

### Academic & Stress Factors
- **CGPA**: Mean 7.66 (SD=1.46)
- **Academic Pressure**: Mean 3.15/5.0
- **Study Satisfaction**: Mean 2.94/5.0
- **Financial Stress**: Mean 3.13/5.0 (highest among segments)

### Lifestyle Patterns
- **Work/Study Hours**: Mean 7.14 hours (highest)
- **Sleep Duration**: Mean 13.61 units
- **Dietary Habits**: More structured patterns

## 2. Worker Population (n=187,794)

### Key Demographics
- **Age**: Mean 44.0 years (SD=11.0)
- **Gender**: 54.7% one gender category
- **Professional Status**: Various professions (mean=42.68)

### Mental Health Profile
- **Depression Rate**: 8.2% (lowest among segments)
- **Suicidal Thoughts**: 46.1%
- **Family History**: 50.1% reported mental illness history

### Work & Stress Factors
- **Work Pressure**: Mean 3.00/5.0
- **Job Satisfaction**: Mean 2.97/5.0
- **Financial Stress**: Mean 2.95/5.0

### Lifestyle Patterns
- **Work/Study Hours**: Mean 6.03 hours
- **Sleep Duration**: Mean 24.40 units
- **Dietary Habits**: More varied patterns

## 3. Unemployed Population (n=33)

### Key Demographics
- **Age**: Mean 31.0 years (SD=11.8)
- **Gender**: 42.4% one gender category
- **Status**: Neither working nor studying

### Mental Health Profile
- **Depression Rate**: 42.1%
- **Suicidal Thoughts**: 63.6%
- **Family History**: 24.2% (lowest among segments)

### Stress Factors
- **Financial Stress**: Mean 3.39/5.0
- **Limited Job/Academic Data**: High missing values

### Lifestyle Patterns
- **Work/Study Hours**: Mean 5.97 hours
- **Sleep Duration**: Mean 1.70 units
- **Dietary Habits**: Limited variation

## Cross-Segment Comparisons

### Depression Rates
1. Students: 58.6%
2. Unemployed: 42.1%
3. Workers: 8.2%

### Risk Factors by Population
- **Students**: Academic pressure, financial stress
- **Workers**: Work pressure, family history
- **Unemployed**: Financial stress, suicidal thoughts

### Protective Factors by Population
- **Students**: Structured routine
- **Workers**: Job stability, lower depression rates
- **Unemployed**: Lower family history of mental illness

## Recommendations

### For Mental Health Professionals
1. **Targeted Interventions**:
   - Students: Academic stress management
   - Workers: Work-life balance programs
   - Unemployed: Financial counseling

2. **Screening Priorities**:
   - High-risk groups (students, unemployed)
   - Suicidal thought assessment
   - Financial stress evaluation

### For Policy Makers
1. **Resource Allocation**:
   - Student mental health services
   - Unemployment support programs
   - Workplace mental health initiatives

2. **Prevention Strategies**:
   - Early intervention programs
   - Financial support systems
   - Mental health awareness campaigns

### Feature Engineering

In [ ]:
# Create visualizations for missing values and data distributions
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set up the style
plt.style.use('default')
sns.set_palette("husl")

# Apply conditional data suppression based on segment criteria
custom_logger("ACTION", "Filtering data based on segment criteria...")

students_filtered = students[
    students['Academic Pressure'].notna() & 
    students['CGPA'].notna() & 
    students['Study Satisfaction'].notna()
].copy()

workers_filtered = workers[
    workers['Work Pressure'].notna() & 
    workers['Job Satisfaction'].notna() & 
    workers['Profession'].notna()
].copy()

unemployed_filtered = unemployed[
    (unemployed['CGPA'].isna()) &
    (unemployed['Work Pressure'].isna()) &
    (unemployed['Job Satisfaction'].isna()) &
    (unemployed['Profession'].isna())
].copy()

# Calculate and log data loss percentages
students_loss = (1 - len(students_filtered)/len(students)) * 100
workers_loss = (1 - len(workers_filtered)/len(workers)) * 100
unemployed_loss = (1 - len(unemployed_filtered)/len(unemployed)) * 100

custom_logger("INFO", f"Data loss after filtering:")
custom_logger("INFO", f"Students: {students_loss:.1f}%")
custom_logger("INFO", f"Workers: {workers_loss:.1f}%") 
custom_logger("INFO", f"Unemployed: {unemployed_loss:.1f}%")

# Create figure for missing values comparison
custom_logger("ACTION", "Generating visualization of missing values...")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Missing values percentages by category after filtering
missing_vals = {
    'Students': {
        'Academic Pressure': (students_filtered['Academic Pressure'].isna().sum() / len(students_filtered)) * 100,
        'CGPA': (students_filtered['CGPA'].isna().sum() / len(students_filtered)) * 100,
        'Study Satisfaction': (students_filtered['Study Satisfaction'].isna().sum() / len(students_filtered)) * 100,
        'Depression': (students_filtered['Depression'].isna().sum() / len(students_filtered)) * 100
    },
    'Workers': {
        'Work Pressure': (workers_filtered['Work Pressure'].isna().sum() / len(workers_filtered)) * 100,
        'Job Satisfaction': (workers_filtered['Job Satisfaction'].isna().sum() / len(workers_filtered)) * 100,
        'Profession': (workers_filtered['Profession'].isna().sum() / len(workers_filtered)) * 100,
        'Depression': (workers_filtered['Depression'].isna().sum() / len(workers_filtered)) * 100
    },
    'Unemployed': {
        'Depression': (unemployed_filtered['Depression'].isna().sum() / len(unemployed_filtered)) * 100,
        'Financial Stress': (unemployed_filtered['Financial Stress'].isna().sum() / len(unemployed_filtered)) * 100
    }
}

# Plot missing values
x_pos = np.arange(len(missing_vals))
for i, (category, values) in enumerate(missing_vals.items()):
    ax1.bar(i, np.mean(list(values.values())), label=f'{category}\n(Avg: {np.mean(list(values.values())):.1f}%)')
    
ax1.set_title('Average Missing Values by Category (After Filtering)', fontsize=14)
ax1.set_ylabel('Percentage Missing (%)')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([k for k in missing_vals.keys()])

# Calculate and plot depression rates for filtered data
custom_logger("ACTION", "Calculating depression rates by category...")

depression_rates = {
    'Students': (students_filtered['Depression'].mean() * 100),
    'Workers': (workers_filtered['Depression'].mean() * 100),
    'Unemployed': (unemployed_filtered['Depression'].mean() * 100)
}

ax2.bar(depression_rates.keys(), depression_rates.values(), color='lightcoral')
ax2.set_title('Depression Rates by Category (After Filtering)', fontsize=14)
ax2.set_ylabel('Depression Rate (%)')

for i, v in enumerate(depression_rates.values()):
    ax2.text(i, v + 1, f'{v:.1f}%', ha='center')

plt.tight_layout()
plt.show()

# Convert filtered DataFrames to Spark DataFrames
custom_logger("ACTION", "Converting filtered data to Spark DataFrames...")

students_spark = spark.createDataFrame(students_filtered)
workers_spark = spark.createDataFrame(workers_filtered)
unemployed_spark = spark.createDataFrame(unemployed_filtered)

# Analyze missing data for each category
custom_logger("INFO", "Detailed Missing Data Analysis by Category:")

categories = {
    'Students': students_filtered,
    'Workers': workers_filtered, 
    'Unemployed': unemployed_filtered
}

# First check for duplicate IDs across categories
all_ids = []
for category, df in categories.items():
    if 'ID' in df.columns:
        all_ids.extend(df['ID'].tolist())

duplicate_ids = set([id for id in all_ids if all_ids.count(id) > 1])

if duplicate_ids:
    custom_logger("WARNING", f"Found {len(duplicate_ids)} duplicate IDs across categories")
    # Remove duplicates keeping first occurrence
    total_removed = 0
    for category, df in categories.items():
        if 'ID' in df.columns:
            initial_size = len(df)
            categories[category] = df[~df['ID'].isin(duplicate_ids)]
            removed_count = initial_size - len(categories[category])
            total_removed += removed_count
            custom_logger("INFO", f"Removed {removed_count} duplicate IDs from {category} category")
    custom_logger("INFO", f"Total of {total_removed} duplicate records removed across all dataframes")

else:
    custom_logger("SUCCESS", "No duplicate IDs found across categories")

# Now proceed with missing value analysis
for category, df in categories.items():
    custom_logger("INFO", f"\n{category} Category:")
    columns_to_drop = []
    for col in df.columns:
        null_count = df[col].isnull().sum()
        if null_count > 0:
            pct_missing = (null_count / len(df)) * 100
            custom_logger("WARNING" if pct_missing > 10 else "INFO", 
                         f"Column '{col}': {null_count} missing values ({pct_missing:.2f}%)")
            if pct_missing > 98:
                columns_to_drop.append(col)
                custom_logger("WARNING", f"Column '{col}' will be dropped due to {pct_missing:.2f}% missing values")
    
    if columns_to_drop:
        categories[category] = df.drop(columns=columns_to_drop)
        # Update the original filtered dataframes with dropped columns
        if category == 'Students':
            students_filtered = categories[category]
        elif category == 'Workers':
            workers_filtered = categories[category]
        elif category == 'Unemployed':
            unemployed_filtered = categories[category]
        custom_logger("INFO", f"Dropped {len(columns_to_drop)} columns from {category} category")

custom_logger("ACTION", "Performing univariate analysis...")

custom_logger("INFO", "Univariate Analysis Results:")

custom_logger("INFO", "Students:")
univariate_analysis(students_filtered)

custom_logger("INFO", "Workers:")
univariate_analysis(workers_filtered)

custom_logger("INFO", "Unemployed:")
univariate_analysis(unemployed_filtered)

# Create final datasets for each population
students_dataset = students_filtered.copy()
workers_dataset = workers_filtered.copy() 
unemployed_dataset = unemployed_filtered.copy()

# Save the datasets
students_dataset.to_csv('students_final.csv', index=False)
workers_dataset.to_csv('workers_final.csv', index=False)
unemployed_dataset.to_csv('unemployed_final.csv', index=False)

custom_logger("SUCCESS", "Created and saved final datasets for all three populations")


In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import (
    Dense, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dropout, BatchNormalization, 
    GlobalMaxPooling1D, Layer, MultiHeadAttention, LayerNormalization, Flatten, Reshape
)
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas as pd
import matplotlib.pyplot as plt

# Disable NCCL
import os
os.environ['NCCL_BLOCKING_WAIT']='0'
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

# ==========================================================================================================================================================#
# ======================================================== GPU MALLOC ======================================================================================#
# ==========================================================================================================================================================#

def freegpu():
    try:
        result = subprocess.run(['sudo', 'fuser', '-v', '/dev/nvidia*'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        for line in result.stdout.decode('utf-8').split('\n'):
            if '/dev/nvidia' in line:
                pid = int(line.split()[-1])
                os.kill(pid, 9)
        print("Freed GPU memory.")
    except Exception as e:
        print(f"Failed to free GPU memory: {e}")

# Define a callback to free GPU memory after each epoch
class FreeGPUMemoryCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        freegpu()

# Clear TensorFlow session
def clear_tf_session():
    clear_session()
    freegpu()

# ==========================================================================================================================================================#
# =========================================================== RESNET BLOCK =================================================================================#
# ==========================================================================================================================================================#

# ================================================================[DENSE]===================================================================================#

# Define a custom block for ResNet with additional layers for better propagation and backpropagation using Dense layers
def resnet_block_dense(x, units, activation='relu'):
    """
    Define a ResNet block using Dense layers with conditional projection and batch normalization.
    
    Parameters:
    - x: Input tensor.
    - units: Number of units in the Dense layers.
    - activation: Activation function to use.

    Returns:
    - A tensor of shape (batch_size, units).

    A shortcut is added to the input tensor to improve the propagation and backpropagation of the signal.
    """
    shortcut = x
    x = Dense(units, activation=activation)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Dense(units, activation=activation)(x)
    x = BatchNormalization()(x)
    shortcut = Dense(units, activation=None)(shortcut)
    x = Add()([x, shortcut])
    x = ReLU()(x)
    return x

# ------------------------------------------------------------------------------------------------------
# Define the strategy with HierarchicalCopyAllReduce to avoid NCCL
# ------------------------------------------------------------------------------------------------------
strategy = tf.distribute.MirroredStrategy(
    devices=["/gpu:0", "/gpu:1"],
    cross_device_ops=tf.distribute.HierarchicalCopyAllReduce()
)

print(f"Number of devices under strategy: {strategy.num_replicas_in_sync}")

class CustomAccuracyMetric(tf.keras.metrics.Metric):
    def __init__(self, name='custom_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.total = self.add_weight(name='total', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Cast inputs to float32 to ensure type consistency
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
        values = tf.cast(tf.equal(y_true, y_pred), tf.float32)
        if sample_weight is not None:
            values = tf.multiply(values, sample_weight)
        self.true_positives.assign_add(tf.reduce_sum(values))
        self.total.assign_add(tf.cast(tf.size(y_true), tf.float32))

    def result(self):
        return self.true_positives / self.total

class HybridTransformerBlock(kt.HyperModel):
    def __init__(self, input_shape):
        super().__init__()
        self.input_shape = input_shape
        
    def build(self, hp):
        # Simplified hyperparameters
        hp_config = {
            # Architecture hyperparameters 
            'cnn_filters': hp.Int('cnn_filters', 16, 64, step=16),
            'kernel_size': hp.Int('kernel_size', 3, 5, step=2),
            'num_heads': hp.Int('num_heads', 2, 4),
            'key_dim': hp.Int('key_dim', 16, 64, step=16),
            'fusion_units': hp.Int('fusion_units', 32, 128, step=32),
            'fusion_units_2': hp.Int('fusion_units_2', 32, 256, step=32),
            'fusion_units_resnet': hp.Int('fusion_units_resnet', 32, 128, step=32),
            
            # Regularization hyperparameters
            'dropout_rate': hp.Float('dropout_rate', 0.1, 0.8, step=0.1),
            'dropout_rate_2': hp.Float('dropout_rate_2', 0.1, 0.8, step=0.1),
            
            # Learning hyperparameters
            'batch_size': hp.Choice('batch_size', [32, 64, 128])
        }

        # Input layer
        inputs = layers.Input(shape=(self.input_shape, 1))
        
        # Simplified CNN Branch
        cnn = layers.Conv1D(
            filters=hp_config['cnn_filters'],
            kernel_size=hp_config['kernel_size'],
            padding='same',
            kernel_regularizer=l1_l2(0.001)
        )(inputs)
        cnn = layers.BatchNormalization()(cnn)
        cnn = layers.LeakyReLU()(cnn)
        
        # Simplified Transformer Branch
        transformer = layers.LayerNormalization(epsilon=1e-6)(inputs)
        attention = MultiHeadAttention(
            num_heads=hp_config['num_heads'],
            key_dim=hp_config['key_dim']
        )(transformer, transformer)
        transformer = layers.Add()([transformer, attention])
        
        # Fusion layer
        fusion = layers.Concatenate()(
            [layers.GlobalAveragePooling1D()(cnn),
             layers.GlobalAveragePooling1D()(transformer)]
        )
        
        # Final dense layer
        x = layers.Dense(hp_config['fusion_units'], activation='relu',
                        kernel_regularizer=l1_l2(0.001))(fusion)
        x = layers.Dropout(hp_config['dropout_rate'])(x)

        for _ in range(2):
            x = resnet_block_dense(x, hp_config['fusion_units_resnet'], activation='relu')
        
        x = layers.Dense(hp_config['fusion_units_2'], activation='relu',
                        kernel_regularizer=l1_l2(0.001))(x)
        x = layers.Dropout(hp_config['dropout_rate_2'])(x)

        outputs = layers.Dense(1, activation='sigmoid')(x)
        
        model = Model(inputs=inputs, outputs=outputs)
        
        # Simple binary crossentropy loss
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model

def preprocess_data(df):
    """Preprocess data while preserving Depression values for training"""
    try:
        custom_logger("INFO", "DATA PREPROCESSING - PROCESS STARTED")

        # Create copy to avoid modifying original
        df_copy = df.copy()
        
        # Store Depression values and their indices
        depression_mask = ~df_copy['Depression'].isna()
        train_indices = df_copy[depression_mask].index
        predict_indices = df_copy[~depression_mask].index
        
        train_depression = df_copy.loc[train_indices, 'Depression']

        # Identify numerical and categorical columns
        numeric_cols = df_copy.select_dtypes(include=['int64', 'float64']).columns
        categorical_cols = df_copy.select_dtypes(include=['object']).columns

        # Remove id and Depression from feature columns
        numeric_cols = [col for col in numeric_cols if col not in ['id', 'Depression']]
        categorical_cols = [col for col in categorical_cols if col not in ['id', 'Depression']]

        # Handle missing values separately for numeric and categorical columns
        for col in numeric_cols:
            df_copy[col] = df_copy[col].fillna(df_copy[col].mean())
            
        for col in categorical_cols:
            df_copy[col] = df_copy[col].fillna(df_copy[col].mode()[0])

        # One-hot encode categorical columns
        df_encoded = pd.get_dummies(df_copy, columns=categorical_cols)

        # Scale numeric features
        scaler = MinMaxScaler()
        df_encoded[numeric_cols] = scaler.fit_transform(df_encoded[numeric_cols])

        # Standardize scaled features
        standard_scaler = StandardScaler()
        df_encoded[numeric_cols] = standard_scaler.fit_transform(df_encoded[numeric_cols])

        # Create feature vector column
        feature_cols = [col for col in df_encoded.columns if col not in ['id', 'Depression']]
        df_encoded['standard_features'] = df_encoded[feature_cols].values.tolist()

        # Split into train and predict sets
        train_data = df_encoded.loc[train_indices].copy()
        train_data['Depression'] = train_depression
        
        predict_data = df_encoded.loc[predict_indices].copy()

        custom_logger("INFO", "DATA PREPROCESSING - PROCESS COMPLETED")
        return train_data, predict_data

    except Exception as e:
        custom_logger("ERROR", f"TASK FAILURE - REASON : {str(e)}")
        raise

# Load datasets
students_df = pd.read_csv('students_final.csv')
workers_df = pd.read_csv('workers_final.csv') 
unemployed_df = pd.read_csv('unemployed_final.csv')

# Store original Depression values before preprocessing
students_original = students_df[~students_df['Depression'].isna()][['id', 'Depression']].copy()
workers_original = workers_df[~workers_df['Depression'].isna()][['id', 'Depression']].copy()
unemployed_original = unemployed_df[~unemployed_df['Depression'].isna()][['id', 'Depression']].copy()

# Preprocess each dataset separately
students_train, students_predict = preprocess_data(students_df)
workers_train, workers_predict = preprocess_data(workers_df)
unemployed_train, unemployed_predict = preprocess_data(unemployed_df)

def train_and_predict(train_data, predict_data, segment_name):
    # Prepare training data
    X = np.array(train_data['standard_features'].tolist())
    y = np.array(train_data['Depression'].tolist()).reshape(-1, 1)
    
    # Cast data to float32
    X = X.astype(np.float32)
    y = y.astype(np.float32)
    
    # Split training data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Reshape for Conv1D
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    
    # Prepare prediction data
    X_predict = np.array(predict_data['standard_features'].tolist())
    X_predict = X_predict.astype(np.float32)
    X_predict = X_predict.reshape((X_predict.shape[0], X_predict.shape[1], 1))
    
    # Create datasets with optimized performance
    AUTOTUNE = tf.data.AUTOTUNE
    BATCH_SIZE = 32
    
    # Training dataset pipeline
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))\
        .cache()\
        .shuffle(buffer_size=64)\
        .batch(BATCH_SIZE)\
        .prefetch(buffer_size=AUTOTUNE)
    
    # Validation dataset pipeline  
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))\
        .cache()\
        .batch(BATCH_SIZE)\
        .prefetch(buffer_size=AUTOTUNE)
    
    # Initialize model builder with input shape
    model_builder = HybridTransformerBlock(input_shape=X_train.shape[1])
    
    callbacks = [
        EarlyStopping(monitor='loss', patience=30, restore_best_weights=True),
    ]
    
    # Configure hyperparameter tuning
    tuner = kt.Hyperband(
        model_builder,
        objective=kt.Objective('val_accuracy', direction='max'),
        directory=f'hyperband_tuning_{segment_name}',
        project_name=f'depression_prediction_{segment_name}'
    )
    
    # Search for best hyperparameters
    tuner.search(train_dataset, epochs=300, validation_data=val_dataset, callbacks=callbacks)
    
    # Get best hyperparameters and build final model
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    
    with strategy.scope():
        model = model_builder.build(best_hps)
    
    # Train final model
    history = model.fit(
        train_dataset,
        epochs=200,
        validation_data=val_dataset,
        callbacks=callbacks
    )
    
    # Make predictions
    predictions = (model.predict(X_predict) > 0.5).astype(int)
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'id': predict_data['id'],
        'Depression': predictions.flatten()
    })
    
    return results_df, history

# Train models and get predictions for each segment
students_predictions, students_history = train_and_predict(students_train, students_predict, 'students')
workers_predictions, workers_history = train_and_predict(workers_train, workers_predict, 'workers')
unemployed_predictions, unemployed_history = train_and_predict(unemployed_train, unemployed_predict, 'unemployed')

# Restore original Depression values
def restore_original_values(predictions_df, original_df):
    merged = predictions_df.merge(original_df, on='id', how='left', suffixes=('_pred', '_orig'))
    merged['Depression'] = merged['Depression_orig'].fillna(merged['Depression_pred'])
    return merged[['id', 'Depression']]

students_predictions = restore_original_values(students_predictions, students_original)
workers_predictions = restore_original_values(workers_predictions, workers_original)
unemployed_predictions = restore_original_values(unemployed_predictions, unemployed_original)

# Combine all predictions
final_predictions = pd.concat([
    students_predictions,
    workers_predictions, 
    unemployed_predictions
], ignore_index=True)

# Save final predictions
final_predictions.to_csv('final_predictions.csv', index=False)

custom_logger("SUCCESS", "Generated and saved final predictions for all segments")